In [1]:
import spacy
import fullmetalalchemy as fa
import tinytable as tt

In [2]:
nlp = spacy.load('en_core_web_sm')

In [3]:
engine = fa.create.create_engine('sqlite:///data.db')
text_table = fa.features.get_table('text', engine)
print(fa.features.get_row_count(text_table))
print(fa.features.get_column_names(text_table))
entity_table = fa.features.get_table('entities', engine)
print(fa.features.get_row_count(entity_table))
print(fa.features.get_column_names(entity_table))
found_table = fa.features.get_table('found_entities', engine)
print(fa.features.get_row_count(found_table))
print(fa.features.get_column_names(found_table))

47959
['id', 'text_id', 'text']
23303
['id', 'entity']
138783
['id', 'text_id', 'entity_id', 'label']


In [ ]:
def insert_label(text_id: int, entity: str, label: str) -> None:
    # check if entity is in entity_table
    existing_entities = fa.select.select_records_by_primary_keys(entity_table, [{'entity': entity}])
    if existing_entities == []:
        # if not, insert into entity_table
        fa.insert.insert_records(entity_table, [{'entity': entity}])
        existing_entities = fa.select.select_records_by_primary_keys(entity_table, [{'entity': entity}])
    entity_id = existing_entities[0]['id']
    # add entity to found_entity table
    record = {'text_id': text_id, 'entity_id': entity_id, 'label': label}
    fa.insert.insert_records(found_table, [record, ])

In [ ]:
for row in fa.select.select_records_all(text_table):
    text_id = row['id']
    doc = nlp(row['text'])
    for ent in doc.ents:
        insert_label(text_id, ent.text, ent.label_)

In [8]:
entities_table = tt.read_sqlite('data.db', 'entities')
entities_table.head(100)

+----+------+----------------------------+
|    |   id | entity                     |
+====+======+============================+
|  0 |    1 | Thousands                  |
+----+------+----------------------------+
|  1 |    2 | London                     |
+----+------+----------------------------+
|  2 |    3 | Iraq                       |
+----+------+----------------------------+
|  3 |    4 | British                    |
+----+------+----------------------------+
|  4 |    5 | Iranian                    |
+----+------+----------------------------+
|  5 |    6 | Wednesday                  |
+----+------+----------------------------+
|  6 |    7 | IAEA                       |
+----+------+----------------------------+
|  7 |    8 | Saturday                   |
+----+------+----------------------------+
|  8 |    9 | Orakzai                    |
+----+------+----------------------------+
|  9 |   10 | Taliban                    |
+----+------+----------------------------+
| 10 |   11

In [9]:
found_table = tt.read_sqlite('data.db', 'found_entities')
found_table.head(100)

+----+------+-----------+-------------+----------+
|    |   id |   text_id |   entity_id | label    |
+====+======+===========+=============+==========+
|  0 |    1 |         0 |           1 | CARDINAL |
+----+------+-----------+-------------+----------+
|  1 |    2 |         0 |           2 | GPE      |
+----+------+-----------+-------------+----------+
|  2 |    3 |         0 |           3 | GPE      |
+----+------+-----------+-------------+----------+
|  3 |    4 |         0 |           4 | NORP     |
+----+------+-----------+-------------+----------+
|  4 |    5 |         1 |           5 | NORP     |
+----+------+-----------+-------------+----------+
|  5 |    6 |         1 |           6 | DATE     |
+----+------+-----------+-------------+----------+
|  6 |    7 |         1 |           7 | ORG      |
+----+------+-----------+-------------+----------+
|  7 |    8 |         2 |           8 | DATE     |
+----+------+-----------+-------------+----------+
|  8 |    9 |         2 |      

In [10]:
entities_table.join(found_table, 'id', 'entity_id', ).head(100)

+----+------+-----------+-----------+----------+-------------+
|    |   id | entity    |   text_id | label    |   entity_id |
+====+======+===========+===========+==========+=============+
|  0 |    1 | Thousands |         0 | CARDINAL |           1 |
+----+------+-----------+-----------+----------+-------------+
|  1 |    1 | Thousands |       212 | CARDINAL |           1 |
+----+------+-----------+-----------+----------+-------------+
|  2 |    1 | Thousands |       318 | CARDINAL |           1 |
+----+------+-----------+-----------+----------+-------------+
|  3 |    1 | Thousands |       419 | CARDINAL |           1 |
+----+------+-----------+-----------+----------+-------------+
|  4 |    1 | Thousands |       523 | CARDINAL |           1 |
+----+------+-----------+-----------+----------+-------------+
|  5 |    1 | Thousands |       623 | CARDINAL |           1 |
+----+------+-----------+-----------+----------+-------------+
|  6 |    1 | Thousands |      1282 | CARDINAL |       